# Create figures related to results for paper Tweet paper

Included plots are:
- 2d histogram on prediction confidence
- confusion matrix
- roc curve

In [ ]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import re
import glob
import zipfile
import pathlib
import logging
import json
import string

import numpy as np
import pandas as pd
import matplotlib.font_manager
import matplotlib.pyplot as plt
import matplotlib

import matplotlib.offsetbox


import wget

import requests
import base64
from bs4 import BeautifulSoup

import wordcloud
import tweepy
import a2.twitter.downloader
import a2.plotting
import a2.dataset
import a2.utils
import a2.training
import a2.plotting.analysis

import joblib

In [ ]:
# set up cache location
location = "./cachedir"
memory = joblib.Memory(location, verbose=1)

In [ ]:
FOLDER_DATA = a2.utils.file_handling.get_folder_data()
FOLDER_TWEETS = FOLDER_DATA / "tweets/"
FILE_TWEETS = (
    FOLDER_TWEETS
    / "2017_2020_tweets_rain_sun_vocab_emojis_locations_bba_Tp_era5_no_bots_normalized_filtered_weather_stations_fix_predicted_simpledeberta_radar.nc"
)
FOLDER_FIGURES = pathlib.Path("../../figures/analysis/modelling/")
FILE_FONT = "../fonts/noto/NotoEmoji-VariableFont_wght.ttf"
FOLDER_MODEL_TO_LOAD = "../../models/model_weights/output_rainprediction_simpledeberta/era5/checkpoint-7617/"
FOLDER_RAW_MODEL = pathlib.Path("../../models/deberta-v3-small/")

In [ ]:
ds_tweets = a2.dataset.load_dataset.load_tweets_dataset(FILE_TWEETS)

In [ ]:
evaluate_hugging_cached = memory.cache(a2.training.evaluate_hugging.make_predictions_loaded_model)

In [ ]:
import pickle

objects = []
with (open("tpp.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [ ]:
(
    truth,
    predictions,
    prediction_probabilities,
) = objects[0]

In [ ]:
# (
#     truth,
#     predictions,
#     prediction_probabilities,
# ) = evaluate_hugging_cached(ds_tweets, ds_tweets.index.values, FOLDER_MODEL_TO_LOAD, folder_tokenizer=FOLDER_RAW_MODEL)

In [ ]:
marginal_x = marginal_y = None
a2.plotting.analysis.plot_prediction_certainty(
    truth,
    prediction_probabilities[:, 1],
    overplot_round_base=-3,
    label_x="Label",
    label_y="Prediction confidence for label 'raining'",
    marginal_x=marginal_x,
    marginal_y=marginal_y,
    marginal_color="navy",
    vmin=0,
    vmax=160000,
    font_size=15,
    overplot_color="snow",
    colormap="cividis",
)
fig = plt.gcf()
fig.savefig(FOLDER_FIGURES / "prediction_certainty_era5.pdf")

In [ ]:
fig, axes, axes_colorbar = a2.plotting.utils_plotting.create_axes_grid(
    2,
    1,
    figure_size=[20, 7],
    unravel=True,
    colorbar_off=False,
    spacing_colorbar=0.02,
    colorbar_width=0.02,
    colorbar_skip_row_col=[[0, 0]],
    spacing_x=0.13,
    left=0.06,
    bottom=0.11,
)
marginal_x = marginal_y = None
font_size = 26
a2.plotting.analysis.plot_roc(truth, prediction_probabilities[:, 1], ax=axes[0], font_size=font_size, fig=fig)
a2.plotting.analysis.plot_confusion_matrix(
    truth,
    predictions,
    ax=axes[1],
    ax_colorbar=axes_colorbar[1],
    overplot_round_base=2,
    text_color="firebrick",
    font_size=font_size,
    vmin=0,
    vmax=0.50,
)
fig.savefig(FOLDER_FIGURES / "confusion_matrix_roc_era5.pdf")